## In this notebook, we will explore various ML algorithms

1. Random forest
2. Naive Bayes
3. Logistic Regression
4. Support Vector Machines
5. Gradient Boosted Classifiers

## Converting text data into forms that ML models can read

Before we utitlize machine learning models for prediction, we need to convert the text data into a form that ML models can read.
Here are some forms we will explore:

1. Word2Vec representation of text (for this, it might be better to not remove stopwords: https://www.kaggle.com/code/harshitmakkar/nlp-word2vec)
2. Bag of Words representation
3. TF-IDF representation

In [2]:
%pip install gensim
%pip install scipy==1.12.0

Note: you may need to restart the kernel to use updated packages.
  Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.12.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.0
    Uninstalling scipy-1.13.0:
      Successfully uninstalled scipy-1.13.0
Note: you may need to restart the kernel to use updated packages.


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.data import find
import gensim
import nltk
from gensim.models import KeyedVectors, Word2Vec

Word2Vec: USing a pre-trained model

In [1]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path = True)

In [2]:
path

'/home/codespace/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz'

In [8]:
# using a pre-trained model which is part of a model that is trained on 100 billion words from the Google News Dataset
# models/word2vec_sample/pruned.word2vec.txt
word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

In [14]:
model